# Running OSTRICH
In this example, we will use mikecalsetup to setup a calibration in OSTRICH and run it. For details on the setup generated by mikecalsetup, the reader is refered to example_usage.ipynb. 

To use OSTRICH you will have to:
- Download Ostrich from http://www.civil.uwaterloo.ca/envmodelling/Ostrich.html.
- Replace the 'C:\bin' in the variable ostpath with the location of your OSTRICH installation.

Lets make a quick calibration setup for an example MIKE SHE model.

In [ ]:
# first some imports
import mikecalsetup
import os
import shutil
os.chdir('../test')
ostpath = 'C:\\bin\\'

In [ ]:
mod_nme = 'Karup_basic0'  # name of model (no extension)
pth = './example_models/Karup/Karup_Basic'  # relative path to model! Full paths not supported!
setup = mikecalsetup.Setup(mod_nme, pth)
par= setup.par
for veg in ['LAI', 'RD', 'Kc']:
    if veg != 'Kc':
        group = [p for p in par.index.tolist() if veg in p and 'Grass' not in p][:3]
        par.loc[group[1:], 'transform'] = 'tied'
        par.loc[group[1:], 'tied_to'] = group[0]
    group = [p for p in par.index.tolist() if veg in p and 'Grass' in p]
    par.loc[group[1:], 'transform'] = 'tied'
    par.loc[group[1:], 'tied_to'] = group[0]
group = [p for p in par.index.tolist() if 'LeakageCoef' in p]
par.loc[group[1:], 'transform'] = 'tied'
par.loc[group[1:], 'tied_to'] = group[0]

par.loc['sz_Drainage_TimeConstant', 'transform'] = 'fixed'
setup.par = par
setup.write_files()
par

With this we should have a working setup. For a test run we should set the number of iterations to 1. We do this by loading the genereated ostin file ad changing the search algorithm configurations.

In [ ]:
ostin = mikecalsetup.OstFile('./ostin.txt')
ostin.load()  # load current setup

# make changes
ostin.configbas['ProgramType'] = 'PADDS'  # change to the basic configuration (section 2.3 in ostin documentation)
ostin.configsa = ['PerturbationValue 0.2', 'MaxIterations 1', 'SelectionMetric Random']  # change to search algorithm configuration (section 2.18-2.20 in ostin documentation)

# write to same ostin file
ostin.write()

In case you have run this example or others several times, lets make sure the working directories do not already exist. These are called ModelSubdir in OSTRICH language (OSTRICH manual page 15). In the ostin file, we have specified that the working directories have the prefix "mod". The following will delete these working directories created by OSTRICH.

In [ ]:
modsubdir = ostin.configbas['ModelSubdir']
old_working_directories = [d for d in os.listdir('.') if d.startswith(modsubdir) and '.' not in d]
old_working_directories

Make sure that this is in fact the working directories before running next cell which will delete these directories.

In [ ]:
modsubdir = ostin.configbas['ModelSubdir']
old_working_directories = [d for d in os.listdir('.') if d.startswith(modsubdir) and '.' not in d]
for directory in old_working_directories:
    shutil.rmtree(directory)

Since the control file for OSTRICH must be called "ostin.txt", the only thing we have to call to run OSTRICH, is the path to OSTRICH. 

In [ ]:
# os.system('cmd /c "'+ostpath+'Ostrich.exe'+'"')

In the OSTRICH documentation it is suggested to write a bat file, which will do the same thing. Double-click the generated bat file to run OSTRICH. 

In [ ]:
with open('OSTRICH - Serial.bat', 'w') as file:
    file.write('@echo off\n\n')
    file.write('"'+ostpath+'Ostrich.exe'+'"\n\n')
    file.write('pause\n')


Double-clicking this bat file will run OSTRICH. OSTRICH will run MIKE SHE one iteration of five evaluations. This is most definitely not enough to calibrate the model, but this is just a quick example make sure the setup works. 

OSTRICH can also run in parallel. Lets make sure the algorithm is compatible with parallel runs.

In [ ]:
ostin = mikecalsetup.OstFile('./ostin.txt')
ostin.load()  # load current setup

# make changes
ostin.configbas['ProgramType'] = 'ParaPADDS'  # change to the basic configuration (section 2.3 in ostin documentation)
ostin.configsa = ['PerturbationValue 0.2', 'MaxIterations 20', 'SelectionMetric Random']  # change to search algorithm configuration (section 2.18-2.20 in ostin documentation)

# write to same ostin file
ostin.write()

Let's also delete the worker folders again.

In [ ]:
old_working_directories = [d for d in os.listdir('.') if d.startswith(modsubdir) and '.' not in d]
old_working_directories

Make sure that this is in fact the working directories before running next cell which will delete these directories.

In [ ]:
modsubdir = ostin.configbas['ModelSubdir']
old_working_directories = [d for d in os.listdir('.') if d.startswith(modsubdir) and '.' not in d]
for directory in old_working_directories:
    shutil.rmtree(directory)

Similarly  to before, we will write a bat file that we can double-click to run OSTRICH. Please note you should specify the desired number of workers here. OSTRICH will in some versions reserve the first worker (in this setup, the directory mod0) for saving of the original setup. Specifying three workers will therefore result in two parallel runs.

In [ ]:
no_of_workers = 3
with open('OSTRICH - Parallel.bat', 'w') as file:
    file.write('@echo off\n\n')
    file.write('mpiexec -n {:d} "'.format(no_of_workers)+ostpath+'OstrichMPI.exe'+'"\n\n')
    file.write('pause\n')